<a href="https://colab.research.google.com/github/raven-gith/machinelearning1/blob/main/12.%20Chapter%2012/chapter_12_custom_model_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 12: Custom Models and Training with TensorFlow

Notebook ini mereproduksi dan menjelaskan isi Bab 12 dari buku _Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow_ oleh Aurélien Géron.

## 📘 Ringkasan Materi:

Bab ini menjelaskan cara membangun model deep learning yang **lebih fleksibel dan dapat dikustomisasi** menggunakan API tingkat rendah TensorFlow.

### Materi utama:
1. Subclassing `tf.keras.Model` dan `tf.keras.layers.Layer`
2. Custom loss, metric, dan training loop (GradientTape)
3. Model fungsional (Functional API)
4. Logging menggunakan TensorBoard


In [1]:

import tensorflow as tf
from tensorflow import keras

# Custom layer
class MyDenseLayer(keras.layers.Layer):
    def __init__(self, units=32, activation=None):
        super().__init__()
        self.units = units
        self.activation = keras.activations.get(activation)

    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer="random_normal",
                                 trainable=True)
        self.b = self.add_weight(shape=(self.units,),
                                 initializer="zeros",
                                 trainable=True)

    def call(self, inputs):
        return self.activation(tf.matmul(inputs, self.w) + self.b)

# Custom model
class MyModel(keras.Model):
    def __init__(self):
        super().__init__()
        self.dense1 = MyDenseLayer(64, activation="relu")
        self.dense2 = MyDenseLayer(10, activation="softmax")

    def call(self, x):
        x = self.dense1(x)
        return self.dense2(x)


In [2]:

# Load dan siapkan data
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()
X_train = X_train.reshape(-1, 28*28) / 255.0
X_test = X_test.reshape(-1, 28*28) / 255.0

# Functional API
inputs = keras.Input(shape=(28*28,))
x = keras.layers.Dense(128, activation="relu")(inputs)
x = keras.layers.Dense(64, activation="relu")(x)
outputs = keras.layers.Dense(10, activation="softmax")(x)
functional_model = keras.Model(inputs=inputs, outputs=outputs)

functional_model.compile(optimizer="adam",
                         loss="sparse_categorical_crossentropy",
                         metrics=["accuracy"])

functional_model.fit(X_train, y_train, epochs=5, validation_split=0.1)


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.8553 - loss: 0.4755 - val_accuracy: 0.9672 - val_loss: 0.1171
Epoch 2/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.9628 - loss: 0.1226 - val_accuracy: 0.9742 - val_loss: 0.0882
Epoch 3/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.9752 - loss: 0.0782 - val_accuracy: 0.9747 - val_loss: 0.0890
Epoch 4/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9826 - loss: 0.0550 - val_accuracy: 0.9768 - val_loss: 0.0811
Epoch 5/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.9861 - loss: 0.0423 - val_accuracy: 0.9743 - val_loss: 0.0865


In [3]:

# Logging menggunakan TensorBoard
import os
log_dir = os.path.join("logs", "fit")
tensorboard_cb = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

functional_model.fit(X_train, y_train, epochs=3, validation_split=0.1, callbacks=[tensorboard_cb])


Epoch 1/3
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9880 - loss: 0.0362 - val_accuracy: 0.9780 - val_loss: 0.0828
Epoch 2/3
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.9903 - loss: 0.0291 - val_accuracy: 0.9748 - val_loss: 0.0923
Epoch 3/3
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.9927 - loss: 0.0213 - val_accuracy: 0.9692 - val_loss: 0.1281
